In [1]:
import pandas as pd
import numpy as np
from yahoo_fin import options
import datetime as dt
from scipy.optimize import minimize

from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols


import warnings

warnings.filterwarnings("ignore")

In [2]:

# Get the options chain for a given ticker
ticker = "^SPX"
spot = 4780.94 # SPX spot price on 2024-01-18
raw_expiration_date = options.get_expiration_dates(ticker)
expiration_dates = raw_expiration_date[:43]

In [3]:
expiration_dates

['January 19, 2024',
 'January 22, 2024',
 'January 23, 2024',
 'January 24, 2024',
 'January 25, 2024',
 'January 26, 2024',
 'January 29, 2024',
 'January 30, 2024',
 'January 31, 2024',
 'February 1, 2024',
 'February 2, 2024',
 'February 5, 2024',
 'February 6, 2024',
 'February 7, 2024',
 'February 8, 2024',
 'February 9, 2024',
 'February 12, 2024',
 'February 13, 2024',
 'February 14, 2024',
 'February 15, 2024',
 'February 16, 2024',
 'February 20, 2024',
 'February 21, 2024',
 'February 23, 2024',
 'February 26, 2024',
 'February 29, 2024',
 'March 1, 2024',
 'March 15, 2024',
 'March 28, 2024',
 'April 19, 2024',
 'April 30, 2024',
 'May 17, 2024',
 'May 31, 2024',
 'June 21, 2024',
 'June 28, 2024',
 'July 19, 2024',
 'August 16, 2024',
 'September 20, 2024',
 'September 30, 2024',
 'October 18, 2024',
 'November 15, 2024',
 'December 20, 2024',
 'December 31, 2024']

In [4]:
chain_dict = {}
# Loop through the expiration data and get the options chain for each date and store it in a dictionary
for dates in expiration_dates:
    chain_dict[dates] = options.get_options_chain(ticker, dates)

df_list = []

# Loop through the dictionary and append each dataframe to the list
for key in chain_dict:
    df_list.append(chain_dict[key]['calls'])

# Use pandas.concat() to concatenate all dataframes into one
merged_df = pd.concat(df_list)

In [5]:
merged_df

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,SPXW240119C00200000,2024-01-18 2:33PM EST,200.0,4572.03,4604.00,4611.00,0.0,-,20,795,"3,643.16%"
1,SPXW240119C00400000,2024-01-17 3:43PM EST,400.0,4331.23,4404.70,4411.80,0.0,-,1,60,"2,730.27%"
2,SPX240119C00600000,2024-01-11 10:37AM EST,600.0,4164.21,0.00,0.00,0.0,-,1,15,0.00%
3,SPX240119C00800000,2024-01-17 3:45PM EST,800.0,3932.69,0.00,0.00,0.0,-,4,39,0.00%
4,SPX240119C01000000,2023-12-26 10:20AM EST,1000.0,3769.25,0.00,0.00,0.0,-,4,4576,0.00%
...,...,...,...,...,...,...,...,...,...,...,...
51,SPXW241231C06000000,2024-01-09 10:15AM EST,6000.0,8.20,8.2,8.70,0.0,-,2,7,12.68%
52,SPXW241231C06100000,2024-01-05 12:55PM EST,6100.0,5.03,5.7,6.00,0.0,-,1,1,12.62%
53,SPXW241231C06200000,2023-12-29 3:19PM EST,6200.0,5.90,3.9,4.30,0.0,-,1,12,12.65%
54,SPXW241231C06400000,2024-01-02 12:41PM EST,6400.0,2.85,2.0,2.30,0.0,-,2,2,12.80%


In [19]:
# Filter the dataframe for the desired strike prices (centered Strikes)
filtered_calls_chain = merged_df[(merged_df['Strike'] >= 4650) & (merged_df['Strike'] <= 4900)]
filtered_calls_chain.reset_index(drop=True, inplace=True)
# remove row that contains 'SPXW' in the 'Contract Name' column
filtered_calls_chain = filtered_calls_chain[~filtered_calls_chain['Contract Name'].str.contains('SPXW')]
filtered_calls_chain.reset_index(drop=True, inplace=True)
filtered_calls_chain

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,SPX240119C04655000,2024-01-18 2:37PM EST,4655.0,119.91,0.00,0.0,0.0,-,2,416,0.00%
1,SPX240119C04660000,2024-01-18 2:37PM EST,4660.0,114.86,0.00,0.0,0.0,-,93,332,0.00%
2,SPX240119C04665000,2024-01-18 3:59PM EST,4665.0,117.87,0.00,0.0,0.0,-,1,433,0.00%
3,SPX240119C04670000,2024-01-18 2:06PM EST,4670.0,89.53,0.00,0.0,0.0,-,114,1245,0.00%
4,SPX240119C04675000,2024-01-18 3:36PM EST,4675.0,105.00,0.00,0.0,0.0,-,127,7232,0.00%
...,...,...,...,...,...,...,...,...,...,...,...
235,SPX241220C04800000,2024-01-18 11:45AM EST,4800.0,344.00,372.1,374.6,0.0,-,226,18820,20.94%
236,SPX241220C04825000,2024-01-17 11:46AM EST,4825.0,318.80,355.3,358.3,0.0,-,23,281,20.66%
237,SPX241220C04850000,2024-01-18 10:08AM EST,4850.0,307.35,339.5,341.7,0.0,-,1,2508,20.35%
238,SPX241220C04875000,2024-01-18 3:54PM EST,4875.0,308.30,323.6,326.0,0.0,-,102,724,20.07%


In [20]:
filtered_calls_chain['Date'] = filtered_calls_chain['Contract Name'].apply(
    lambda x: x[3:9] if len(x) == 18 else (x[4:10] if len(x) == 19 else x))

filtered_calls_chain['Date'] = pd.to_datetime(
    filtered_calls_chain['Date'], format='%y%m%d')

filtered_calls_chain.set_index("Date", inplace=True)

In [21]:
filtered_calls_chain

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
Date,,,,,,,,,,,
2024-01-19,SPX240119C04655000,2024-01-18 2:37PM EST,4655.0,119.91,0.00,0.0,0.0,-,2,416,0.00%
2024-01-19,SPX240119C04660000,2024-01-18 2:37PM EST,4660.0,114.86,0.00,0.0,0.0,-,93,332,0.00%
2024-01-19,SPX240119C04665000,2024-01-18 3:59PM EST,4665.0,117.87,0.00,0.0,0.0,-,1,433,0.00%
2024-01-19,SPX240119C04670000,2024-01-18 2:06PM EST,4670.0,89.53,0.00,0.0,0.0,-,114,1245,0.00%
2024-01-19,SPX240119C04675000,2024-01-18 3:36PM EST,4675.0,105.00,0.00,0.0,0.0,-,127,7232,0.00%
...,...,...,...,...,...,...,...,...,...,...,...
2024-12-20,SPX241220C04800000,2024-01-18 11:45AM EST,4800.0,344.00,372.1,374.6,0.0,-,226,18820,20.94%
2024-12-20,SPX241220C04825000,2024-01-17 11:46AM EST,4825.0,318.80,355.3,358.3,0.0,-,23,281,20.66%
2024-12-20,SPX241220C04850000,2024-01-18 10:08AM EST,4850.0,307.35,339.5,341.7,0.0,-,1,2508,20.35%


In [22]:
# replace all "_" with 0 in the dataframe
filtered_calls_chain = filtered_calls_chain.replace('-', 0)
# strike, last price, bid, ask to float
filtered_calls_chain['Strike'] = filtered_calls_chain['Strike'].astype(float)
filtered_calls_chain['Last Price'] = filtered_calls_chain['Last Price'].astype(float)
filtered_calls_chain['Bid'] = filtered_calls_chain['Bid'].astype(float)
filtered_calls_chain['Ask'] = filtered_calls_chain['Ask'].astype(float)

In [23]:
filtered_calls_chain['price'] = (filtered_calls_chain['Bid'] + filtered_calls_chain['Ask']) / 2

In [24]:
# remove the rows with no bid or ask price
filtered_calls_chain = filtered_calls_chain[filtered_calls_chain['price'] != 0]
filtered_calls_chain

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility,price
Date,,,,,,,,,,,,
2024-02-16,SPX240216C04650000,2024-01-18 2:11PM EST,4650.0,154.50,190.5,192.7,0.0,0,157,8847,21.81%,191.60
2024-02-16,SPX240216C04655000,2024-01-18 2:37PM EST,4655.0,159.95,185.6,188.3,0.0,0,4,67,21.58%,186.95
2024-02-16,SPX240216C04670000,2024-01-18 3:53PM EST,4670.0,154.40,172.6,175.3,0.0,0,8,905,20.91%,173.95
2024-02-16,SPX240216C04675000,2024-01-18 1:37PM EST,4675.0,125.06,168.4,171.0,0.0,0,6,3552,20.69%,169.70
2024-02-16,SPX240216C04680000,2024-01-18 9:40AM EST,4680.0,126.66,164.1,166.7,0.0,0,2,393,20.46%,165.40
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-20,SPX241220C04800000,2024-01-18 11:45AM EST,4800.0,344.00,372.1,374.6,0.0,0,226,18820,20.94%,373.35
2024-12-20,SPX241220C04825000,2024-01-17 11:46AM EST,4825.0,318.80,355.3,358.3,0.0,0,23,281,20.66%,356.80
2024-12-20,SPX241220C04850000,2024-01-18 10:08AM EST,4850.0,307.35,339.5,341.7,0.0,0,1,2508,20.35%,340.60


In [25]:
df_calls = filtered_calls_chain[['Strike', 'price', 'Implied Volatility']]
df_calls

,Strike,price,Implied Volatility
Date,,,
2024-02-16,4650.0,191.60,21.81%
2024-02-16,4655.0,186.95,21.58%
2024-02-16,4670.0,173.95,20.91%
2024-02-16,4675.0,169.70,20.69%
2024-02-16,4680.0,165.40,20.46%
...,...,...,...
2024-12-20,4800.0,373.35,20.94%
2024-12-20,4825.0,356.80,20.66%
2024-12-20,4850.0,340.60,20.35%


In [26]:
# Pivot the dataframe to get the strike prices as columns and the dates as rows
df_calls_pivot = df_calls.pivot(columns="Strike", values="price")

In [27]:
df_calls_pivot

Strike,4650.0,4655.0,4660.0,4665.0,4670.0,4675.0,4680.0,4685.0,4690.0,4695.0,...,4855.0,4860.0,4865.0,4870.0,4875.0,4880.0,4885.0,4890.0,4895.0,4900.0
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-16,191.60,186.95,NaN,NaN,173.95,169.70,165.40,161.20,157.00,152.65,...,NaN,NaN,NaN,38.15,NaN,34.10,32.30,NaN,NaN,26.90
2024-03-15,223.75,219.65,215.50,211.7,NaN,202.95,199.40,NaN,190.85,187.10,...,77.7,NaN,NaN,NaN,NaN,NaN,NaN,60.35,58.05,55.85
2024-04-19,NaN,NaN,NaN,251.5,246.95,243.30,239.30,235.45,231.55,227.60,...,117.1,114.25,111.2,108.50,105.65,102.85,100.45,NaN,NaN,NaN
2024-05-17,291.30,NaN,NaN,NaN,275.70,271.50,229.20,NaN,NaN,NaN,...,NaN,143.30,NaN,137.40,134.20,NaN,NaN,NaN,NaN,NaN
2024-06-21,323.70,NaN,316.35,NaN,308.25,304.85,301.05,NaN,293.25,NaN,...,NaN,176.10,NaN,170.05,166.75,164.00,NaN,158.10,NaN,152.25
2024-07-19,350.15,NaN,NaN,NaN,NaN,330.50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,193.05,NaN,NaN,NaN,NaN,177.55
2024-08-16,374.70,NaN,NaN,NaN,NaN,355.20,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,218.00,NaN,NaN,NaN,NaN,202.70
2024-09-20,403.15,NaN,NaN,NaN,NaN,384.90,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,247.45,NaN,NaN,NaN,NaN,231.45
2024-10-18,425.75,NaN,NaN,NaN,NaN,407.45,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,270.70,NaN,NaN,NaN,NaN,255.10


In [28]:
# remove the strike column that has all more than 0 nan values
df_calls_pivot_t = df_calls_pivot.dropna(axis=1, thresh=1)
df_calls_pivot_t

Strike,4650.0,4655.0,4660.0,4665.0,4670.0,4675.0,4680.0,4685.0,4690.0,4695.0,...,4855.0,4860.0,4865.0,4870.0,4875.0,4880.0,4885.0,4890.0,4895.0,4900.0
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-16,191.60,186.95,NaN,NaN,173.95,169.70,165.40,161.20,157.00,152.65,...,NaN,NaN,NaN,38.15,NaN,34.10,32.30,NaN,NaN,26.90
2024-03-15,223.75,219.65,215.50,211.7,NaN,202.95,199.40,NaN,190.85,187.10,...,77.7,NaN,NaN,NaN,NaN,NaN,NaN,60.35,58.05,55.85
2024-04-19,NaN,NaN,NaN,251.5,246.95,243.30,239.30,235.45,231.55,227.60,...,117.1,114.25,111.2,108.50,105.65,102.85,100.45,NaN,NaN,NaN
2024-05-17,291.30,NaN,NaN,NaN,275.70,271.50,229.20,NaN,NaN,NaN,...,NaN,143.30,NaN,137.40,134.20,NaN,NaN,NaN,NaN,NaN
2024-06-21,323.70,NaN,316.35,NaN,308.25,304.85,301.05,NaN,293.25,NaN,...,NaN,176.10,NaN,170.05,166.75,164.00,NaN,158.10,NaN,152.25
2024-07-19,350.15,NaN,NaN,NaN,NaN,330.50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,193.05,NaN,NaN,NaN,NaN,177.55
2024-08-16,374.70,NaN,NaN,NaN,NaN,355.20,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,218.00,NaN,NaN,NaN,NaN,202.70
2024-09-20,403.15,NaN,NaN,NaN,NaN,384.90,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,247.45,NaN,NaN,NaN,NaN,231.45
2024-10-18,425.75,NaN,NaN,NaN,NaN,407.45,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,270.70,NaN,NaN,NaN,NaN,255.10


In [18]:
# remove strike prices with more than 0 NaN values
df_calls_pivot_t = df_calls_pivot.dropna(axis=1)
df_calls_pivot_t

Strike,4675.0
Date,
2024-02-16,169.70
2024-03-15,202.95
2024-04-19,243.30
2024-05-17,271.50
2024-06-21,304.85
2024-07-19,330.50
2024-08-16,355.20
2024-09-20,384.90
2024-10-18,407.45


In [28]:
# Calculate the time to expiration in years for each date and set time to expiration as index
df_calls_pivot['Time to Expiration'] = (
    df_calls_pivot.index - dt.datetime.now()).days / 365.25  # type: ignore
df_calls_pivot.set_index('Time to Expiration', inplace=True)

In [29]:
# set columns names to numeric
df_calls_pivot.columns = df_calls_pivot.columns.astype(float)

In [30]:
df_calls_pivot = df_calls_pivot.iloc[(df_calls_pivot.index >= 0) & (df_calls_pivot.index <= 1), (df_calls_pivot.columns > 4600) & (df_calls_pivot.columns < 5000)] # type: ignore

In [31]:
df_calls_pivot

Strike,4750.0,4755.0,4760.0,4765.0,4770.0,4775.0,4780.0,4785.0,4790.0,4795.0,...,4855.0,4860.0,4865.0,4870.0,4875.0,4880.0,4885.0,4890.0,4895.0,4900.0
Time to Expiration,,,,,,,,,,,,,,,,,,,,,
0.005476,52.30,48.10,43.80,39.60,35.50,31.80,28.20,24.80,21.70,18.40,...,1.425,1.075,0.75,0.625,0.475,0.35,0.300,0.25,0.20,0.150
0.008214,55.00,50.90,46.70,42.75,39.05,35.25,31.75,28.55,25.45,22.50,...,2.850,2.250,1.80,1.400,1.125,0.85,0.675,0.55,0.45,0.350
0.010951,57.70,53.50,49.70,45.90,42.15,38.55,35.05,31.95,28.85,25.95,...,4.600,3.800,3.15,2.575,2.100,1.70,1.375,1.10,0.90,0.700
0.016427,64.50,60.55,57.05,52.90,49.30,45.85,42.45,39.30,36.15,33.20,...,8.800,7.650,6.75,5.750,4.950,4.15,3.650,3.10,2.70,2.225
0.030116,74.75,71.20,67.65,64.15,60.65,57.15,54.05,50.65,47.75,44.80,...,17.650,16.100,14.65,13.250,11.950,10.85,9.550,8.85,7.85,7.050
0.035592,81.70,77.90,74.40,71.00,67.55,64.15,60.95,57.70,54.75,51.65,...,23.100,21.400,19.70,18.200,16.700,15.35,14.050,12.85,11.70,10.750
0.073922,102.60,99.05,95.65,92.25,88.65,85.45,82.05,78.95,75.75,72.65,...,41.000,38.800,36.80,34.000,32.850,30.30,28.500,27.45,25.85,23.650
0.109514,118.55,115.05,111.45,108.00,104.65,101.20,97.95,94.65,91.45,88.25,...,54.950,52.600,50.30,48.050,45.850,43.75,41.750,39.90,37.85,35.950
0.150582,137.65,134.00,130.60,127.00,123.65,120.25,116.90,113.65,110.85,107.05,...,72.250,70.250,67.85,65.300,62.950,60.60,58.300,55.45,53.30,51.200


In [33]:
# Convert our vol surface to dataframe for each option price with parameters
volSurfaceLong = df_calls_pivot.melt(ignore_index=False).reset_index()
volSurfaceLong.columns = ["maturity", "strike", "price"]
volSurfaceLong

,maturity,strike,price
0,0.005476,4750.0,52.30
1,0.008214,4750.0,55.00
2,0.010951,4750.0,57.70
3,0.016427,4750.0,64.50
4,0.030116,4750.0,74.75
...,...,...,...
305,0.035592,4900.0,10.75
306,0.073922,4900.0,23.65
307,0.109514,4900.0,35.95
308,0.150582,4900.0,51.20


### BBG Data

In [49]:
# load csv form data folder
volSurfaceLong_t = pd.read_excel(r'C:\Users\rrenard\Arkus_python\pricing_library\data\Book1.xlsx', index_col=0, parse_dates=True, header=0)
# pivot
df_calls_pivot_t = volSurfaceLong_t.pivot(columns="Strike", values="Price")

#time to expiration
df_calls_pivot_t['Time to Expiration'] = (
    df_calls_pivot_t.index - dt.datetime.now()).days / 365.25  # type: ignore
df_calls_pivot_t.set_index('Time to Expiration', inplace=True)

# Convert our vol surface to dataframe for each option price with parameters
volSurfaceLong = df_calls_pivot_t.melt(ignore_index=False).reset_index()
volSurfaceLong.columns = ["maturity", "strike", "price"]
volSurfaceLong


,maturity,strike,price
0,-0.002738,4720,90.200000
1,0.030116,4720,109.000000
2,0.073922,4720,133.050000
3,0.106776,4720,184.650000
4,0.109514,4720,149.250000
...,...,...,...
138,0.246407,4825,134.500000
139,0.276523,4825,145.550000
140,0.323066,4825,164.050000
141,0.361396,4825,177.050000


In [50]:
yield_maturities = np.array([1/12, 2/12, 3/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
yeilds = np.array([5.52, 5.54, 5.53, 5.41, 5.27, 4.9, 4.68, 4.52, 4.56, 4.53, 4.87, 4.68]).astype(float)/100

In [51]:
# NSS model calibrate
curve_fit, status = calibrate_nss_ols(yield_maturities, yeilds)

curve_fit

NelsonSiegelSvenssonCurve(beta0=0.04982862243152366, beta1=0.006715571191473596, beta2=-0.014602722209638338, beta3=-0.00888301212085435, tau1=2.0, tau2=5.0)

In [52]:
# Calculate the risk free rate for each maturity using the fitted yield curve
volSurfaceLong['rate'] = volSurfaceLong['maturity'].apply(
    curve_fit)  # type: ignore

In [53]:
volSurfaceLong

,maturity,strike,price,rate
0,-0.002738,4720,90.200000,0.056544
1,0.030116,4720,109.000000,0.056358
2,0.073922,4720,133.050000,0.056093
3,0.106776,4720,184.650000,0.055898
4,0.109514,4720,149.250000,0.055882
...,...,...,...,...
138,0.246407,4825,134.500000,0.055106
139,0.276523,4825,145.550000,0.054943
140,0.323066,4825,164.050000,0.054695
141,0.361396,4825,177.050000,0.054496


In [96]:
# Define variables to be used in optimization
S0 = 4780.94
r = volSurfaceLong['rate'].to_numpy('float')
K = volSurfaceLong['strike'].to_numpy('float')
tau = volSurfaceLong['maturity'].to_numpy('float')
P = volSurfaceLong['price'].to_numpy('float')

params = {"v0": {"x0": 0.1, "lbub": [0,0.1]},
          "kappa": {"x0": 5, "lbub": [1e-3,5]},
          "theta": {"x0": 0.1, "lbub": [1e-3,0.1]},
          "sigma": {"x0": 1, "lbub": [1e-2,1]},
          "rho": {"x0": 1, "lbub": [-1,1]},
          "lambd": {"x0": 1, "lbub": [-1,1]},
          }

x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]

In [97]:
from scipy.integrate import quad
# Heston characteristic function


def heston_charfunc(
    phi,
    initial_stock_price,
    initial_variance,
    kappa,
    theta,
    sigma,
    rho,
    lambd,
    tau,
    risk_free_rate,
):
    # constants
    a = kappa * theta
    b = kappa + lambd
    # common terms w.r.t phi
    rspi = rho * sigma * phi * 1j
    # define d parameter given phi and b
    d = np.sqrt((rho * sigma * phi * 1j - b) **
                2 + (phi * 1j + phi**2) * sigma**2)
    # define g parameter given phi, b and d
    g = (b - rspi + d) / (b - rspi - d)
    # calculate characteristic function by components
    exp1 = np.exp(risk_free_rate * phi * 1j * tau)
    term2 = initial_stock_price ** (phi * 1j) * (
        (1 - g * np.exp(d * tau)) / (1 - g)
    ) ** (-2 * a / sigma**2)
    exp2 = np.exp(
        a * tau * (b - rspi + d) / sigma**2
        + initial_variance
        * (b - rspi + d)
        * ((1 - np.exp(d * tau)) / (1 - g * np.exp(d * tau)))
        / sigma**2
    )
    return exp1 * term2 * exp2


# Heston integrand
def integrand(
    phi,
    initial_stock_price,
    strike,
    initial_variance,
    kappa,
    theta,
    sigma,
    rho,
    lambd,
    tau,
    risk_free_rate,
):
    args = (
        initial_stock_price,
        initial_variance,
        kappa,
        theta,
        sigma,
        rho,
        lambd,
        tau,
        risk_free_rate,
    )
    numerator = np.exp(risk_free_rate * tau) * heston_charfunc(
        phi - 1j, *args
    ) - strike * heston_charfunc(phi, *args)
    denominator = 1j * phi * strike ** (1j * phi)
    return numerator / denominator


# Heston price using rectangular integration
def heston_price_rec(
    initial_stock_price,
    strike,
    initial_variance,
    kappa,
    theta,
    sigma,
    rho,
    lambd,
    tau,
    risk_free_rate,
):
    args = (
        initial_stock_price,
        initial_variance,
        kappa,
        theta,
        sigma,
        rho,
        lambd,
        tau,
        risk_free_rate,
    )

    P, umax, N = 0, 100, 10000
    dphi = umax / N  # dphi is width

    for i in range(1, N):
        # rectangular integration
        phi = dphi * (2 * i + 1) / 2  # midpoint to calculate height
        numerator = np.exp(risk_free_rate * tau) * heston_charfunc(
            phi - 1j, *args
        ) - strike * heston_charfunc(phi, *args)
        denominator = 1j * phi * strike ** (1j * phi)
        P += dphi * numerator / denominator
    call = np.real(
        (initial_stock_price - strike * np.exp(-risk_free_rate * tau)) / 2 + P / np.pi
    )
    put = call - initial_stock_price + strike * np.exp(-risk_free_rate * tau)
    return [call, put]


# Heston price using quadrature integration
def heston_price(
    initial_stock_price,
    strike,
    initial_variance,
    kappa,
    theta,
    sigma,
    rho,
    lambd,
    tau,
    risk_free_rate,
):
    args = (
        initial_stock_price,
        strike,
        initial_variance,
        kappa,
        theta,
        sigma,
        rho,
        lambd,
        tau,
        risk_free_rate,
    )

    real_integral, err = np.real(quad(integrand, 0, 100, args=args))
    call = (
        initial_stock_price - strike * np.exp(-risk_free_rate * tau)
    ) / 2 + real_integral / np.pi
    put = call - initial_stock_price + strike * np.exp(-risk_free_rate * tau)
    return [call, put, err]

In [98]:
def SqErr(x):
    v0, kappa, theta, sigma, rho, lambd = [param for param in x]
    err = np.sum( (P-heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r))**2 /len(P) )

    return err

In [99]:
result = minimize(SqErr, x0, tol=1e-3, method='SLSQP',options={'maxiter': 1e5}, bounds=bnds)

In [100]:
result

 message: Inequality constraints incompatible
 success: False
  status: 4
     fun: nan
       x: [ 1.000e+00  2.000e+01  1.000e+00  1.000e+00  1.000e+00
            1.000e+00]
     nit: 1
     jac: [       nan        nan        nan        nan        nan
                  nan]
    nfev: 7
    njev: 1

In [101]:
v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
v0, kappa, theta, sigma, rho, lambd

(1.0, 20.0, 1.0, 1.0, 1.0, 1.0)

In [102]:
heston_prices = heston_price_rec(
    S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)

In [103]:
volSurfaceLong["heston_price"] = heston_prices[0]

In [104]:
volSurfaceLong['abs_diff'] = abs(
    volSurfaceLong['price'] - volSurfaceLong['heston_price'])

In [105]:
volSurfaceLong

,maturity,strike,price,rate,heston_price,abs_diff
0,-0.002738,4720,90.200000,0.056544,-8.900038e+06,8.900129e+06
1,0.030116,4720,109.000000,0.056358,3.621073e+02,2.531073e+02
2,0.073922,4720,133.050000,0.056093,5.509600e+02,4.179100e+02
3,0.106776,4720,184.650000,0.055898,6.573118e+02,4.726618e+02
4,0.109514,4720,149.250000,0.055882,6.654131e+02,5.161631e+02
...,...,...,...,...,...,...
138,0.246407,4825,134.500000,0.055106,9.494378e+02,8.149378e+02
139,0.276523,4825,145.550000,0.054943,1.008996e+03,8.634457e+02
140,0.323066,4825,164.050000,0.054695,1.095472e+03,9.314217e+02
141,0.361396,4825,177.050000,0.054496,1.162487e+03,9.854365e+02


## Other Calibration found

In [15]:
from scipy.optimize import broyden1
X0 = 100
V0 = 0.2
r = 0.05
kappa = 1.5768
theta=0.0398
lambd=0.575
rho=-0.5711
def heston(kappa,theta,lambd,T,K):
    I=complex(0,1)
    P, umax, N = 0, 1000, 10000
    du=umax/N
    aa= theta*kappa*T/lambd**2
    bb= -2*theta*kappa/lambd**2
    for i in range (1,N) :
        u2=i*du
        u1=complex(u2,-1)
        a1=rho*lambd*u1*I
        a2=rho*lambd*u2*I
        d1=np.sqrt((a1-kappa)**2+lambd**2*(u1*I+u1**2))
        d2=np.sqrt((a2-kappa)**2+lambd**2*(u2*I+u2**2))
        g1=(kappa-a1-d1)/(kappa-a1+d1)
        g2=(kappa-a2-d2)/(kappa-a2+d2)
        b1=np.exp(u1*I*(np.log(X0/K)+r*T))*( (1-g1*np.exp(-d1*T))/(1-g1) )**bb
        b2=np.exp(u2*I*(np.log(X0/K)+r*T))*( (1-g2*np.exp(-d2*T))/(1-g2) )**bb
        phi1=b1*np.exp(aa*(kappa-a1-d1)\
        +V0*(kappa-a1-d1)*(1-np.exp(-d1*T))/(1-g1*np.exp(-d1*T))/lambd**2)
        phi2=b2*np.exp(aa*(kappa-a2-d2)\
        +V0*(kappa-a2-d2)*(1-np.exp(-d2*T))/(1-g2*np.exp(-d2*T))/lambd**2)
        P+= ((phi1-phi2)/(u2*I))*du
    return K*np.real((X0/K-np.exp(-r*T))/2+P/np.pi)
# Example of usage of heston()
T,K=1,100
call = heston(kappa,theta,lambd,T,K)
print("call = ",call, " put = ", call-X0+K*np.exp(-r*T))
# example of calibration
price1=heston(kappa,theta,lambd,T,90)
price2=heston(kappa,theta,lambd,T,105)
price3=heston(kappa,theta,lambd,T,110)

def F(x):
    return [(price1-heston(x[0],x[1],x[2],T,90)), \
    (price2-heston(x[0],x[1],x[2],T,105)), \
    (price3-heston(x[0],x[1],x[2],T,110))]
x = broyden1(F, [3,0.2,0.5], f_tol=1e-9)
print("[kappa,theta,lambda] =",x)

call =  15.788558109615058  put =  10.91150055968646


NoConvergence: [3.  0.2 0.5]

In [3]:
import sys
sys.path.append(
    r"C:\Users\rrenard\Arkus_python\pricing_library\models\options")

from black_scholes import BlackScholes

In [ ]:
BlackScholes(100, 100, 0.05, 1, 0.2).call_price

6.040088129724239